In [ ]:
! pip install plotly matplotlib

In [ ]:
import plotly.express as px
from skimage import io
import numpy as np


image_files = [

    "/content/SP1.5Na60_NC_25%GLU_6Glass slide_0170.JPG",
    "/content/SP1.5Na60_NC_25%GLU_6Glass slide_0171.JPG",
    "/content/SP1.5Na60_NC_25%GLU_6Glass slide_0172.JPG"


]
images = np.array([io.imread(f) for f in image_files])


fig = px.imshow(
    images,
    animation_frame=0,
    labels=dict(animation_frame="Magnetic Field (T)"),
    title="Reorientation of Myelin Figures under Magnetic Field"
)
fig.update_layout(width=600, height=600)
fig.layout.sliders[0]['steps'] = [
    {'label': '0 T', 'method': 'animate', 'args': [[0]]},
    {'label': '0.03 T', 'method': 'animate', 'args': [[1]]},
    {'label': '0.05 T', 'method': 'animate', 'args': [[2]]}

]

fig.write_html("myelin_animation.html")
fig.write_json("myelin_animation.json")


fig.write_html("temp.html")
import plotly.io as pio
pio.write_html(fig, file='myelin_animation.html', auto_open=False)


In [ ]:
!pip install gdown




In [ ]:
path='/content/25-20251107T181726Z-1-001.zip'

In [ ]:
! pip install scikit-image opencv-python numpy pandas matplotlib

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, glob, os
from skimage import io, color, filters, morphology
from skimage.morphology import remove_small_objects, skeletonize
from scipy.ndimage import distance_transform_edt, gaussian_filter as gaussian
from scipy.stats import spearmanr, linregress
from scipy.optimize import curve_fit


img_list = [
    '/content/SP1.5Na60_NC_25_GLU_6Glass slide_0189.JPG',
    '/content/SP1.5Na60_NC_25_GLU_6Glass slide_0190.JPG',
    '/content/SP1.5Na60_NC_25_GLU_6Glass slide_0191.JPG',
    '/content/SP1.5Na60_NC_25_GLU_6Glass slide_0192.JPG',
    '/content/SP1.5Na60_NC_25_GLU_6Glass slide_0193.JPG'
]


def load_image(path, as_gray=True):
    img = io.imread(path)
    if img.ndim == 3 and as_gray:
        img = color.rgb2gray(img)
    return img.astype(np.float32)

def preprocess(img, gaussian_sigma=1.0, clip_percentile=(1,99)):
    img_s = gaussian(img, sigma=gaussian_sigma)
    lo, hi = np.percentile(img_s, clip_percentile)
    img_cs = np.clip((img_s - lo) / (hi - lo + 1e-12), 0, 1)
    return img_cs

def orientation_map(img, sigma=3):
    from skimage.feature import structure_tensor
    Axx, Axy, Ayy = structure_tensor(img, sigma=sigma)
    theta = 0.5 * np.arctan2(2 * Axy, Axx - Ayy)
    tmp = np.sqrt((Axx - Ayy)**2 + 4 * Axy**2)
    l1 = 0.5 * (Axx + Ayy + tmp)
    l2 = 0.5 * (Axx + Ayy - tmp)
    coherence = (l1 - l2) / (l1 + l2 + 1e-12)
    return theta, coherence

def nematic_order(theta_pixels):
    doubled = 2 * theta_pixels
    C = np.mean(np.cos(doubled))
    S_sin = np.mean(np.sin(doubled))
    theta0_double = np.arctan2(S_sin, C)
    theta0 = 0.5 * theta0_double
    S = np.sqrt(C**2 + S_sin**2)
    return float(S), float(theta0)

def compute_mean_diameter(binary_mask, pixel_size_um=0.1, min_obj_size=200):
    cleaned = remove_small_objects(binary_mask, min_size=min_obj_size)
    skel = skeletonize(cleaned)
    if skel.sum() == 0:
        return np.nan, None
    dist = distance_transform_edt(cleaned)
    diameters_um = 2.0 * dist[skel] * pixel_size_um
    diameters_um = diameters_um[diameters_um > 0]
    if len(diameters_um) == 0:
        return np.nan, None
    return float(np.mean(diameters_um)), diameters_um

def analyze_image(path, pixel_size_um=0.1):
    img = load_image(path)
    proc = preprocess(img, gaussian_sigma=1.0)
    thresh = filters.threshold_otsu(proc)
    mask = proc > thresh * 0.6
    mask = morphology.binary_closing(mask, morphology.disk(3))
    mask = morphology.binary_opening(mask, morphology.disk(2))
    theta_map, coherence = orientation_map(proc, sigma=2)
    valid = (coherence > 0.2) & mask
    if valid.sum() < 10:
        valid = mask
    S, director = nematic_order(theta_map[valid])
    mean_d, _ = compute_mean_diameter(mask, pixel_size_um=pixel_size_um)
    return {"path": path, "S_parameter": S, "mean_diameter_um": mean_d}

# --- Run analysis ---
results = [analyze_image(p, pixel_size_um=0.16) for p in img_list]
df = pd.DataFrame(results)
print("✅ Analysis done.")
display(df)

plt.figure(figsize=(6,4))
plt.hist(df["S_parameter"].dropna(), bins=5)
plt.xlabel("S parameter")
plt.ylabel("Count")
plt.title("Distribution of Nematic Order (S)")
plt.show()


df["field_T"] =
df["time_s"] =


plt.figure(figsize=(6,4))
plt.plot(df["field_T"], df["S_parameter"], 'o-')
plt.xlabel("Magnetic Field (T)")
plt.ylabel("S parameter")
plt.title("S vs Magnetic Field")
plt.show()


slope, intercept, r_val, p_val, _ = linregress(df["field_T"], df["S_parameter"])
rho, p_spear = spearmanr(df["field_T"], df["S_parameter"])
print(f"Linear R² = {r_val**2:.3f}, Spearman ρ = {rho:.3f} (p={p_spear:.3e})")
